In [1]:
import h2o
import pandas as pd
import numpy as np

In [2]:
h2o.init()
h2o.remove_all() 



No instance found at ip and port: localhost:54321. Trying to start local jar...


JVM stdout: /tmp/tmpPdnOKN/h2o_david_started_from_python.out
JVM stderr: /tmp/tmphhycI0/h2o_david_started_from_python.err
Using ice_root: /tmp/tmp6ELYC2


Java Version: java version "1.7.0_91"
OpenJDK Runtime Environment (IcedTea 2.6.3) (7u91-2.6.3-0ubuntu0.14.04.1)
OpenJDK 64-Bit Server VM (build 24.91-b01, mixed mode)


Starting H2O JVM and connecting: ............... Connection successful!


H2O cluster uptime:,1 seconds 765 milliseconds
H2O cluster version:,3.6.0.3
H2O cluster name:,H2O_started_from_python
H2O cluster total nodes:,1
H2O cluster total memory:,1.72 GB
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


In [3]:
df = h2o.import_file("../data/raw/train_users.csv")


Parse Progress: [##################################################] 100%


In [4]:
df = df.drop('id')

In [6]:
X = df.col_names[:-1]
del X[2]

y = df.col_names[-1]

In [81]:
# from h2o.grid.grid_search import H2OGridSearch

# ntrees_opt = 1
# max_depth_opt = [2]
# learn_rate_opt = [0.1,0.2]
# hyper_parameters = {"ntrees": ntrees_opt, "max_depth":max_depth_opt, "learn_rate":learn_rate_opt, "distribution":"multinomial"}

# gs = H2OGridSearch(H2OGradientBoostingEstimator, hyper_params=hyper_parameters, )

# gs.train(X, y, training_frame=df)
# print "R2:", dl.r2()


gbm Grid Build Progress: [##################################################] 100%


### Deep Learning Model

In [88]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator

dl = H2OGradientBoostingEstimator(ntrees=43,
                                  max_depth=6, 
                                  distribution='multinomial',
                                  stopping_metric="logloss",
                                  learn_rate=0.25,
                                  sample_rate=0.6,
                                  balance_classes=True, 
                                  nfolds=0)

In [ ]:
dl.train(X, y, training_frame=df)


gbm Model Build Progress: [                                                  ] 00%

In [85]:
print "R2:", dl.r2()

R2:

TypeError: 'NoneType' object has no attribute '__getitem__'

Best: 0.929497476375

dl = H2OGradientBoostingEstimator(ntrees=43,
                                  max_depth=6, 
                                  stopping_metric="misclassification",
                                  balance_classes=True, 
                                  nfolds=0)

### Predictions

In [77]:
test = h2o.import_file("../data/raw/test_users.csv")


Parse Progress: [##################################################] 100%


In [78]:
pred = dl.predict(test.drop('id'))

TypeError: 'NoneType' object is not iterable

### Make CSV for submission

In [34]:
p_df = pred.as_data_frame()
p_df = pd.DataFrame(p_df)
p_df = p_df.transpose()
p_df.columns = p_df.iloc[0]
p_df = p_df.reindex(p_df.index.drop(0))
p_df.drop('predict', axis=1, inplace=True)

In [35]:
ids = pd.DataFrame(test.as_data_frame())
ids = ids.transpose()
ids.columns = ids.iloc[0]
ids = ids.reindex(ids.index.drop(0))

In [36]:
A = []
for item in xrange(p_df.shape[0]):
    s = p_df.iloc[item]
    s = s.astype(float)
    s = s.sort_values(ascending=False)
    for i in xrange(5):
        user_id = ids.iloc[item]['id']
        dest = list(s.index[0:5])[i]
        A.append([user_id, dest])

In [37]:
sub = pd.DataFrame(A, columns = ['id', 'country'])

Write CSV file:

In [38]:
sub.to_csv('../data/submissions/sub_h2o.csv',index=False)

In [ ]:
h2o.shutdown()